This is data analysis for study of Bakardjian's method in 

In [1]:

import aseegg as ag
import numpy as np
import pandas as pd
from bieg import ICAManager
import matplotlib.pyplot as plt
import math
import scipy.signal as sig
import math
from sklearn.preprocessing import Normalizer


In [74]:
#Preparing data

import aseegg as ag
import numpy as np
import pandas as pd
from bieg import ICAManager
import matplotlib.pyplot as plt
import math
import scipy.signal as sig
import math
from sklearn.preprocessing import Normalizer

pre_data_path = "../data/subject1/SSVEP_8Hz_Trial1_SUBJ1.MAT"

class BakardjianSystem(object):
    
    def __init__(self,path,seconds,components_to_exclude = [0,14,16,22,24,27,28,127],s_08Hz=np.zeros(1),
                s_14Hz = np.zeros(1),s_28Hz = np.zeros(1)):
        self.path = path
        self.seconds = seconds
        self.ica_file = ICAManager(input_path=self.path,method='fastica')
        self.components_to_exclude = components_to_exclude
        self.s_08Hz = s_08Hz
        self.s_14Hz = s_14Hz
        self.s_28Hz = s_28Hz

    #Module 1: Blind Source Separation/ICA 
    def bss_ica(self):
        
        self.ica_file = ICAManager(input_path=self.path,method='fastica')
        self.ica_file.load_matlab_data()
        self.ica_file.extract_components()
        self.ica_file.exclude_ica_components(components_to_exclude=self.components_to_exclude)
        return self.ica_file
        
    def bank_of_filters(self):
    
        #Electrodes of occipital region
        
        ica_file = self.ica_file
        
        beg = 5*256
        end = 20*256

        electrode_O1 = ica_file.data[15][beg:end]
        electrode_0z = ica_file.data[23][beg:end]
        electrode_O2 = ica_file.data[28][beg:end]
        #Module 2: Narrow-band filters

        prefiltered = [electrode_0z,electrode_O1,electrode_O2]

        #Signals filtered:

        self.s_08Hz = np.zeros((3,len(electrode_O1)))
        self.s_14Hz = np.zeros((3,len(electrode_O1)))
        self.s_28Hz = np.zeros((3,len(electrode_O1)))

        for n in range(0,3):
            signal_filtered_08Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=7.9,czestOdciecia2=8.1,czestProbkowania=256)
            signal_filtered_14Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=13.9,czestOdciecia2=14.1,czestProbkowania=256)
            signal_filtered_28Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=27.9,czestOdciecia2=28.1,czestProbkowania=256)
            self.s_08Hz[n] = signal_filtered_08Hz
            self.s_14Hz[n] = signal_filtered_14Hz
            self.s_28Hz[n] = signal_filtered_28Hz
            
        return self.s_08Hz, self.s_14Hz, self.s_28Hz
        
    #Module3: Variance analyzer

    def variance_analyzing(array_to_va):
        squared = np.square(array_to_va)
        result = np.sqrt(squared)
        return result
    
    def variance_analyzer(self):
        for z in [self.s_08Hz,self.s_14Hz,self.s_28Hz]:
            for n in range(0,z.shape[0]):
                z[n] = self.variance_analyzing(z[n])
        return self.s_08Hz, self.s_14Hz, self.s_28Hz

    #Module 4: smoother

    def smoothing(array_to_smooth,window_length=15):
        smoothed = sig.savgol_filter(array_to_smooth,polyorder=2,window_length=window_length)
        return smoothed
    
    def smoother(self):        
        for n in [self.s_08Hz,self.s_14Hz,self.s_28Hz]:
            for z in range(0,n.shape[0]):
                n[z] = smoothing(n[z],window_length=15)
        return self.s_08Hz,self.s_14Hz,self.s_28Hz

    #Module5: integrator

    def integrating(self,array_to_integrate):
        result = 0
        length = array_to_integrate.shape[0]
        for n in range(0,length):
            result += array_to_integrate[n]
        result = result/length
        return results
    
    def integrator(self):      
        for n in [self.s_08Hz,self.s_14Hz,self.s_28Hz]:
            n = integrating(n)
        return self.s_08Hz,self.s_14Hz,self.s_28Hz

    """    
    #Module6: Normalizer

    normalized = []

    for r in integrated:
        result = Normalizer(r)
        normalized.append(result)
    """

In [75]:
bs = BakardjianSystem(pre_data_path,seconds=15)

In [76]:
bs.bss_ica()

Creating RawArray with float64 data, n_channels=128, n_times=6330
    Range : 0 ... 6329 =      0.000 ...    24.723 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected


/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)
[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 128
<ICA  |  raw data decomposition, fit (fastica): 2052 samples, 128 components, channels used: "eeg">
Creating RawArray with float64 data, n_channels=128, n_times=6330
    Range : 0 ... 6329 =      0.000 ...    24.723 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected


/usr/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 128


[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


<ICA  |  raw data decomposition, fit (fastica): 2052 samples, 128 components, channels used: "eeg">
Transforming to ICA space (128 components)
Zeroing out 8 ICA components


In [77]:
bs.bank_of_filters()

(array([[ -3.79214511e-01,  -5.31407481e-01,  -6.63150842e-01, ...,
           3.07768482e-06,   2.50833605e-06,   1.92874170e-06],
        [ -3.15242093e-01,  -4.52938249e-01,  -5.73186224e-01, ...,
           2.82553979e-06,   2.33146164e-06,   1.81959206e-06],
        [ -3.68288954e-01,  -5.04944005e-01,  -6.22147273e-01, ...,
           2.99920363e-06,   2.44372328e-06,   1.87845966e-06]]),
 array([[  5.42252663e-02,  -6.42360788e-02,  -1.75459833e-01, ...,
           5.96140890e-08,   1.24924349e-07,   1.59607539e-07],
        [ -1.15786206e-02,  -8.88005578e-02,  -1.55847603e-01, ...,
           2.21637412e-08,   7.50600380e-08,   1.06362531e-07],
        [  5.12567075e-02,  -2.40265752e-02,  -9.67200005e-02, ...,
           4.05561558e-08,   1.32765514e-07,   1.87125983e-07]]),
 array([[  8.62034006e-02,   2.01640636e-01,   2.25729721e-01, ...,
          -8.73625199e-08,   1.62066170e-08,   8.82222520e-08],
        [  9.98810644e-02,   1.92301778e-01,   1.97518396e-01, ...,
    

In [78]:
bs.variance_analyzer()

TypeError: variance_analyzing() takes 1 positional argument but 2 were given

In [ ]:
#Module 2: Narrow-band filters

prefiltered = [electrode_0z,electrode_O1,electrode_O2]

#Signals filtered:

s_08Hz = np.zeros((3,len(signal_filtered_08Hz)))
s_14Hz = np.zeros((3,len(signal_filtered_08Hz)))
s_28Hz = np.zeros((3,len(signal_filtered_08Hz)))

for n in range(0,3):
    signal_filtered_08Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=7.9,czestOdciecia2=8.1,czestProbkowania=256)
    signal_filtered_14Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=13.9,czestOdciecia2=14.1,czestProbkowania=256)
    signal_filtered_28Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=27.9,czestOdciecia2=28.1,czestProbkowania=256)
    s_08Hz[n] = signal_filtered_08Hz
    s_14Hz[n] = signal_filtered_14Hz
    s_28Hz[n] = signal_filtered_28Hz

#Module3: Variance analyzer
    
def variance_analyzer(array_to_va):
    squared = np.square(array_to_va)
    result = np.sqrt(squared)
    return result

for z in [s_08Hz,s_14Hz,s_28Hz]:
    for n in range(0,z.shape[0]):
        z[n] = variance_analyzer(z[n])

#Module 4: smoother

def smoothing(array_to_smooth,window_length=15):
    smoothed = sig.savgol_filter(array_to_smooth,polyorder=2,window_length=window_length)
    return smoothed

for n in [s_08Hz,s_14Hz,s_28Hz]:
    for z in range(0,n.shape[0]):
        n[z] = smoothing(n[z],window_length=15)
    
#Module5: integrator

def integrating(array_to_integrate):
    result = 0
    length = array_to_integrate.shape[0]
    for n in range(0,length):
        result += array_to_integrate[n]
    result = result/length
    return results

for n in [s_08Hz,s_14Hz,s_28Hz]:
    n = integrating(n)

"""    
#Module6: Normalizer

normalized = []

for r in integrated:
    result = Normalizer(r)
    normalized.append(result)
"""